In [8]:
from main_extractor import *
from text_feature import *
from utils.dict_lib import *

fs = [TextFeature("rank", "mean_key"), TextFeature("words", "select_keys_proportion", args=("я", "ты"))]

d = text2fеatures("1813\\3", fs)
print(d)

d = important_feature("words", n_top=30, rang=(2, 5))
print(d)
print(len(d))

d = important_feature("n_grams_letter", n_top=40, rang=(3, 5))
print(d)
print(len(d))

[0.1004481418852913, 0.03561643835616438, 0.03561643835616438]
{'друг': 4, 'о': 4, 'любовь': 3, 'вы': 5, 'всё': 2, 'рука': 2, 'весь': 5, 'а': 5, 'где': 4, 'душа': 3, 'день': 2, 'от': 2, 'они': 5, 'его': 2, 'то': 5, 'да': 2, 'так': 3, 'кто': 2, 'нет': 2, 'тот': 2}
20
{'нны': 4, 'дру': 5, 'рас': 4, 'лен': 4, 'руг': 4, 'ель': 3, 'люб': 4, 'вет': 4, 'тра': 3, 'под': 5, 'что': 5, 'нас': 3, 'нов': 5, 'нье': 4, 'его': 5, 'льн': 3, 'как': 5, 'сво': 3, 'раз': 4, 'ово': 5, 'мен': 3, 'одн': 3, 'она': 4}
23
